In [1]:
import os
import re
import shutil
import random

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms, datasets, utils
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.model_selection import KFold
import itertools

pd.set_option('display.max_columns', 1000)

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('BindingDB_All_202402.tsv', delimiter='\t', on_bad_lines='warn')

/var/folders/kk/6q2jd0hj56z_vzsn0nsr_bf40000gn/T/ipykernel_45129/2101285070.py:1: ParserWarning: Skipping line 1356263: expected 194 fields, saw 206
Skipping line 1356289: expected 194 fields, saw 206

  df = pd.read_csv('BindingDB_All_202402.tsv', delimiter='\t', on_bad_lines='warn')
/var/folders/kk/6q2jd0hj56z_vzsn0nsr_bf40000gn/T/ipykernel_45129/2101285070.py:1: ParserWarning: Skipping line 1388430: expected 194 fields, saw 206

  df = pd.read_csv('BindingDB_All_202402.tsv', delimiter='\t', on_bad_lines='warn')
/var/folders/kk/6q2jd0hj56z_vzsn0nsr_bf40000gn/T/ipykernel_45129/2101285070.py:1: ParserWarning: Skipping line 1411094: expected 194 fields, saw 266

  df = pd.read_csv('BindingDB_All_202402.tsv', delimiter='\t', on_bad_lines='warn')
/var/folders/kk/6q2jd0hj56z_vzsn0nsr_bf40000gn/T/ipykernel_45129/2101285070.py:1: ParserWarning: Skipping line 1814863: expected 194 fields, saw 242

  df = pd.read_csv('BindingDB_All_202402.tsv', delimiter='\t', on_bad_lines='warn')
/var/folders

In [3]:
df

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),kon (M-1-s-1),koff (s-1),pH,Temp (C),Curation/DataSource,Article DOI,BindingDB Entry DOI,PMID,PubChem AID,Patent Number,Authors,Institution,Link to Ligand in BindingDB,Link to Target in BindingDB,Link to Ligand-Target Pair in BindingDB,Ligand HET ID in PDB,PDB ID(s) for Ligand-Target Complex,PubChem CID,PubChem SID,ChEBI ID of Ligand,ChEMBL ID of Ligand,DrugBank ID of Ligand,IUPHAR_GRAC ID of Ligand,KEGG ID of Ligand,ZINC ID of Ligand,Number of Protein Chains in Target (>1 implies a multichain complex),BindingDB Target Chain Sequence,PDB ID(s) of Target Chain,UniProt (SwissProt) Recommended Name of Target Chain,UniProt (SwissProt) Entry Name of Target Chain,UniProt (SwissProt) Primary ID of Target Chain,UniProt (SwissProt) Secondary ID(s) of Target Chain,UniProt (SwissProt) Alternative ID(s) of Target Chain,UniProt (TrEMBL) Submitted Name of Target Chain,UniProt (TrEMBL) Entry Name of Target Chain,UniProt (TrEMBL) Primary ID of Target Chain,UniProt (TrEMBL) Secondary ID(s) of Target Chain,UniProt (TrEMBL) Alternative ID(s) of Target Chain,BindingDB Target Chain Sequence.1,PDB ID(s) of Target Chain.1,UniProt (SwissProt) Recommended Name of Target Chain.1,UniProt (SwissProt) Entry Name of Target Chain.1,UniProt (SwissProt) Primary ID of Target Chain.1,UniProt (SwissProt) Secondary ID(s) of Target Chain.1,UniProt (SwissProt) Alternative ID(s) of Target Chain.1,UniProt (TrEMBL) Submitted Name of Target Chain.1,UniProt (TrEMBL) Entry Name of Target Chain.1,UniProt (TrEMBL) Primary ID of Target Chain.1,UniProt (TrEMBL) Secondary ID(s) of Target Chain.1,UniProt (TrEMBL) Alternative ID(s) of Target Chain.1,BindingDB Target Chain Sequence.2,PDB ID(s) of Target Chain.2,UniProt (SwissProt) Recommended Name of Target Chain.2,UniProt (SwissProt) Entry Name of Target Chain.2,UniProt (SwissProt) Primary ID of Target Chain.2,UniProt (SwissProt) Secondary ID(s) of Target Chain.2,UniProt (SwissProt) Alternative ID(s) of Target Chain.2,UniProt (TrEMBL) Submitted Name of Target Chain.2,UniProt (TrEMBL) Entry Name of Target Chain.2,UniProt (TrEMBL) Primary ID of Target Chain.2,UniProt (TrEMBL) Secondary ID(s) of Target Chain.2,UniProt (TrEMBL) Alternative ID(s) of Target Chain.2,BindingDB Target Chain Sequence.3,PDB ID(s) of Target Chain.3,UniProt (SwissProt) Recommended Name of Target Chain.3,UniProt (SwissProt) Entry Name of Target Chain.3,UniProt (SwissProt) Primary ID of Target Chain.3,UniProt (SwissProt) Secondary ID(s) of Target Chain.3,UniProt (SwissProt) Alternative ID(s) of Target Chain.3,UniProt (TrEMBL) Submitted Name of Target Chain.3,UniProt (TrEMBL) Entry Name of Target Chain.3,UniProt (TrEMBL) Primary ID of Target Chain.3,UniProt (TrEMBL) Secondary ID(s) of Target Chain.3,UniProt (TrEMBL) Alternative ID(s) of Target Chain.3,BindingDB Target Chain Sequence.4,PDB ID(s) of Target Chain.4,UniProt (SwissProt) Recommended Name of Target Chain.4,UniProt (SwissProt) Entry Name of Target Chain.4,UniProt (SwissProt) Primary ID of Target Chain.4,UniProt (SwissProt) Secondary ID(s) of Target Chain.4,UniProt (SwissProt) Alternative ID(s) of Target Chain.4,UniProt (TrEMBL) Submitted Name of Target Chain.4,UniProt (TrEMBL) Entry Name of Target Chain.4,UniProt (TrEMBL) Primary ID of Target Chain.4,UniProt (TrEMBL) Secondary ID(s) of Target Chain.4,UniProt (TrEMBL) Alternative ID(s) of Target Chain.4,BindingDB Target Chain Sequence.5,PDB ID(s) of Target Chain.5,UniProt (SwissProt) Recommended Name of Target Chain.5,UniProt (SwissProt) Entry Name of Target Chain.5,UniProt (SwissProt) Primary ID of Target Chain.5,UniProt (SwissProt) Secondary ID(s) of Target Chain.5,UniProt (SwissProt) Alternative ID(s) of Target Chain.5,UniProt (TrEMBL) Submitted Name of Target Chain.5,UniProt (TrEMBL) Entry Name of Target Chain.5,UniProt (TrEMBL) Prim

In [4]:
print([col for col in df.columns if 'seq' in col.lower()])

['BindingDB Target Chain Sequence', 'BindingDB Target Chain Sequence.1', 'BindingDB Target Chain Sequence.2', 'BindingDB Target Chain Sequence.3', 'BindingDB Target Chain Sequence.4', 'BindingDB Target Chain Sequence.5', 'BindingDB Target Chain Sequence.6', 'BindingDB Target Chain Sequence.7', 'BindingDB Target Chain Sequence.8', 'BindingDB Target Chain Sequence.9', 'BindingDB Target Chain Sequence.10', 'BindingDB Target Chain Sequence.11', 'BindingDB Target Chain Sequence.12']


In [5]:
df['BindingDB Target Chain Sequence']

0          PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1          PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2          PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
3          PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
4          PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
                                 ...                        
2830520    MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...
2830521    MEQPQEEAPEVREEEEKEEVAEAEGAPELNGGPQHALPSSSYTDLS...
2830522    MEQPQEEAPEVREEEEKEEVAEAEGAPELNGGPQHALPSSSYTDLS...
2830523    MDTKHFLPLDFSTQVNSSLTSPTGRGSMAAPSLHPSLGPGIGSPGQ...
2830524    MDTKHFLPLDFSTQVNSSLTSPTGRGSMAAPSLHPSLGPGIGSPGQ...
Name: BindingDB Target Chain Sequence, Length: 2830525, dtype: object

In [6]:
df.loc[0,'BindingDB Target Chain Sequence']

'PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNF'

In [7]:
print([col for col in df.columns if 'target' in col.lower()])

['Target Name', 'Target Source Organism According to Curator or DataSource', 'Link to Target in BindingDB', 'Link to Ligand-Target Pair in BindingDB', 'PDB ID(s) for Ligand-Target Complex', 'Number of Protein Chains in Target (>1 implies a multichain complex)', 'BindingDB Target Chain Sequence', 'PDB ID(s) of Target Chain', 'UniProt (SwissProt) Recommended Name of Target Chain', 'UniProt (SwissProt) Entry Name of Target Chain', 'UniProt (SwissProt) Primary ID of Target Chain', 'UniProt (SwissProt) Secondary ID(s) of Target Chain', 'UniProt (SwissProt) Alternative ID(s) of Target Chain', 'UniProt (TrEMBL) Submitted Name of Target Chain', 'UniProt (TrEMBL) Entry Name of Target Chain', 'UniProt (TrEMBL) Primary ID of Target Chain', 'UniProt (TrEMBL) Secondary ID(s) of Target Chain', 'UniProt (TrEMBL) Alternative ID(s) of Target Chain', 'BindingDB Target Chain Sequence.1', 'PDB ID(s) of Target Chain.1', 'UniProt (SwissProt) Recommended Name of Target Chain.1', 'UniProt (SwissProt) Entry Na

In [8]:
pd.isna(df).sum()

BindingDB Reactant_set_id                                      0
Ligand SMILES                                                  0
Ligand InChI                                              116412
Ligand InChI Key                                          116412
BindingDB MonomerID                                            0
                                                          ...   
UniProt (TrEMBL) Submitted Name of Target Chain.12       2830525
UniProt (TrEMBL) Entry Name of Target Chain.12           2830525
UniProt (TrEMBL) Primary ID of Target Chain.12           2830525
UniProt (TrEMBL) Secondary ID(s) of Target Chain.12      2830525
UniProt (TrEMBL) Alternative ID(s) of Target Chain.12    2830525
Length: 194, dtype: int64

In [12]:
count_na = pd.isna(df).sum() 

for col in tqdm(df.columns):
    if count_na[col] == df.shape[0]:
        df.drop(col, axis=1, inplace=True)

100%|██████████| 194/194 [21:54<00:00,  6.78s/it]


In [14]:
df.shape

(2830525, 136)

## Preprocessing

### Filter the target source organism

In [15]:
df['Target Source Organism According to Curator or DataSource'].value_counts(normalize=True)

Target Source Organism According to Curator or DataSource
Homo sapiens                                                          8.114833e-01
Rattus norvegicus                                                     5.956589e-02
Mus musculus                                                          2.624375e-02
Rattus                                                                1.831645e-02
Human immunodeficiency virus 1                                        1.178042e-02
                                                                          ...     
Escherichia coli O6                                                   5.445327e-07
Human cytomegalovirus                                                 5.445327e-07
Haemophilus influenzae (strain ATCC 51907 / DSM 11121 / KW20 / Rd)    5.445327e-07
Aeromonas allosaccharophila                                           5.445327e-07
Bovine viral diarrhea virus (strain CP7)                              5.445327e-07
Name: proportion, Length: 320

In [16]:
df_pre = df[df['Target Source Organism According to Curator or DataSource'].isin(['Homo sapiens','Rattus norvegicus','Mus musculus','Bos taurus'])].copy()

In [17]:
df_pre['Target Source Organism According to Curator or DataSource'].value_counts(normalize=True)

Target Source Organism According to Curator or DataSource
Homo sapiens         0.894786
Rattus norvegicus    0.065681
Mus musculus         0.028938
Bos taurus           0.010595
Name: proportion, dtype: float64

In [18]:
df_pre.shape

(1665468, 136)

### Filter the IC50 value

In [20]:
print([col for col in df_pre.columns if 'ic50' in col.lower()])

['IC50 (nM)']


In [22]:
pd.isna(df_pre['IC50 (nM)']).sum()

603356

In [21]:
print([col for col in df_pre.columns if 'kd' in col.lower()])

['Kd (nM)']


In [23]:
pd.isna(df_pre['Kd (nM)']).sum()

1598245

In [24]:
print([col for col in df_pre.columns if 'ec50' in col.lower()])

['EC50 (nM)']


In [25]:
pd.isna(df_pre['EC50 (nM)']).sum()

1520243

In [27]:
df_pre['IC50_coalesce_intermediate'] = np.where(pd.isna(df_pre['IC50 (nM)']), df_pre['Kd (nM)'], df_pre['IC50 (nM)'])
df_pre['IC50_coalesce'] = np.where(pd.isna(df_pre['IC50_coalesce_intermediate']), df_pre['EC50 (nM)'], df_pre['IC50_coalesce_intermediate'])

In [30]:
df_pre['IC50_coalesce']

142         >50000
198            NaN
199            NaN
200            NaN
201            NaN
            ...   
2830518    50800.0
2830519       50.0
2830520     6700.0
2830523       26.0
2830524    52900.0
Name: IC50_coalesce, Length: 1665468, dtype: object

In [34]:
df_pre['IC50_coalesce'].value_counts()

IC50_coalesce
>10000     75060
<100       27488
>100000    22484
>1000      15963
<10        11905
           ...  
 0.1806        1
 0.2215        1
 0.1115        1
 0.0384        1
52900.0        1
Name: count, Length: 45239, dtype: int64

In [37]:
# Simple preprocessing: remove the inequality sign

df_pre['IC50_coalesce_pre'] = np.where(df_pre['IC50_coalesce'].str[0].isin(['<','>']), df_pre['IC50_coalesce'].str[1:],df_pre['IC50_coalesce'])

In [41]:
df_pre['IC50_coalesce_pre'] = df_pre['IC50_coalesce_pre'].astype(float)

In [42]:
df_pre2 = df_pre[df_pre['IC50_coalesce_pre'] <= 100].copy()

In [43]:
df_pre2.shape

(487642, 139)

### Has a chemical identifier

In [44]:
print([col for col in df_pre2.columns if 'cid' in col.lower()])

['PubChem CID']


In [46]:
pd.isna(df_pre2['PubChem CID']).sum()

5107

In [47]:
df_pre3 = df_pre2[~pd.isna(df_pre2['PubChem CID'])].copy()

### Every record have SMILES representation

In [48]:
pd.isna(df_pre3['Ligand SMILES']).sum()

0

### Molecular weight <= 1000Da

In [52]:
print([col for col in df_pre3.columns if 'mass' in col.lower()])

[]


### The molecule has a protein identifier

In [53]:
print([col for col in df_pre3.columns if 'uniprot' in col.lower()])

['UniProt (SwissProt) Recommended Name of Target Chain', 'UniProt (SwissProt) Entry Name of Target Chain', 'UniProt (SwissProt) Primary ID of Target Chain', 'UniProt (SwissProt) Secondary ID(s) of Target Chain', 'UniProt (TrEMBL) Submitted Name of Target Chain', 'UniProt (TrEMBL) Entry Name of Target Chain', 'UniProt (TrEMBL) Primary ID of Target Chain', 'UniProt (TrEMBL) Secondary ID(s) of Target Chain', 'UniProt (SwissProt) Recommended Name of Target Chain.1', 'UniProt (SwissProt) Entry Name of Target Chain.1', 'UniProt (SwissProt) Primary ID of Target Chain.1', 'UniProt (SwissProt) Secondary ID(s) of Target Chain.1', 'UniProt (TrEMBL) Submitted Name of Target Chain.1', 'UniProt (TrEMBL) Entry Name of Target Chain.1', 'UniProt (TrEMBL) Primary ID of Target Chain.1', 'UniProt (TrEMBL) Secondary ID(s) of Target Chain.1', 'UniProt (SwissProt) Recommended Name of Target Chain.2', 'UniProt (SwissProt) Entry Name of Target Chain.2', 'UniProt (SwissProt) Primary ID of Target Chain.2', 'UniP

In [55]:
pd.isna(df_pre3['UniProt (SwissProt) Primary ID of Target Chain']).sum()

639

In [56]:
df_pre4 = df_pre3[~pd.isna(df_pre3['UniProt (SwissProt) Primary ID of Target Chain'])].copy()

In [57]:
df_pre4.shape

(481896, 139)

### Amino acid sequence length >= 80 and <= 2050 

In [61]:
df_pre4['TargetChainSequenceLength'] = df_pre4['BindingDB Target Chain Sequence'].apply(lambda x: len(x))

In [62]:
df_pre4['TargetChainSequenceLength']

1960        292
2879       1132
3038       1132
3129        671
3322        351
           ... 
2830480      99
2830481      99
2830482      99
2830519     505
2830523     462
Name: TargetChainSequenceLength, Length: 481896, dtype: int64

In [63]:
df_pre5 = df_pre4[(df_pre4['TargetChainSequenceLength'] <= 2050) & (df_pre4['TargetChainSequenceLength'] >= 80)].copy()

In [64]:
df_pre5.shape

(470505, 140)

## Remove the unused columns

In [65]:
df_pre5

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),kon (M-1-s-1),koff (s-1),pH,Temp (C),Curation/DataSource,Article DOI,BindingDB Entry DOI,PMID,PubChem AID,Patent Number,Authors,Institution,Link to Ligand in BindingDB,Link to Target in BindingDB,Link to Ligand-Target Pair in BindingDB,Ligand HET ID in PDB,PDB ID(s) for Ligand-Target Complex,PubChem CID,PubChem SID,ChEBI ID of Ligand,ChEMBL ID of Ligand,DrugBank ID of Ligand,IUPHAR_GRAC ID of Ligand,KEGG ID of Ligand,ZINC ID of Ligand,Number of Protein Chains in Target (>1 implies a multichain complex),BindingDB Target Chain Sequence,PDB ID(s) of Target Chain,UniProt (SwissProt) Recommended Name of Target Chain,UniProt (SwissProt) Entry Name of Target Chain,UniProt (SwissProt) Primary ID of Target Chain,UniProt (SwissProt) Secondary ID(s) of Target Chain,UniProt (TrEMBL) Submitted Name of Target Chain,UniProt (TrEMBL) Entry Name of Target Chain,UniProt (TrEMBL) Primary ID of Target Chain,UniProt (TrEMBL) Secondary ID(s) of Target Chain,BindingDB Target Chain Sequence.1,PDB ID(s) of Target Chain.1,UniProt (SwissProt) Recommended Name of Target Chain.1,UniProt (SwissProt) Entry Name of Target Chain.1,UniProt (SwissProt) Primary ID of Target Chain.1,UniProt (SwissProt) Secondary ID(s) of Target Chain.1,UniProt (TrEMBL) Submitted Name of Target Chain.1,UniProt (TrEMBL) Entry Name of Target Chain.1,UniProt (TrEMBL) Primary ID of Target Chain.1,UniProt (TrEMBL) Secondary ID(s) of Target Chain.1,BindingDB Target Chain Sequence.2,PDB ID(s) of Target Chain.2,UniProt (SwissProt) Recommended Name of Target Chain.2,UniProt (SwissProt) Entry Name of Target Chain.2,UniProt (SwissProt) Primary ID of Target Chain.2,UniProt (SwissProt) Secondary ID(s) of Target Chain.2,UniProt (TrEMBL) Submitted Name of Target Chain.2,UniProt (TrEMBL) Entry Name of Target Chain.2,UniProt (TrEMBL) Primary ID of Target Chain.2,UniProt (TrEMBL) Secondary ID(s) of Target Chain.2,BindingDB Target Chain Sequence.3,PDB ID(s) of Target Chain.3,UniProt (SwissProt) Recommended Name of Target Chain.3,UniProt (SwissProt) Entry Name of Target Chain.3,UniProt (SwissProt) Primary ID of Target Chain.3,UniProt (SwissProt) Secondary ID(s) of Target Chain.3,UniProt (TrEMBL) Submitted Name of Target Chain.3,UniProt (TrEMBL) Entry Name of Target Chain.3,UniProt (TrEMBL) Primary ID of Target Chain.3,BindingDB Target Chain Sequence.4,PDB ID(s) of Target Chain.4,UniProt (SwissProt) Recommended Name of Target Chain.4,UniProt (SwissProt) Entry Name of Target Chain.4,UniProt (SwissProt) Primary ID of Target Chain.4,UniProt (SwissProt) Secondary ID(s) of Target Chain.4,UniProt (TrEMBL) Submitted Name of Target Chain.4,UniProt (TrEMBL) Entry Name of Target Chain.4,UniProt (TrEMBL) Primary ID of Target Chain.4,BindingDB Target Chain Sequence.5,PDB ID(s) of Target Chain.5,UniProt (SwissProt) Recommended Name of Target Chain.5,UniProt (SwissProt) Entry Name of Target Chain.5,UniProt (SwissProt) Primary ID of Target Chain.5,UniProt (SwissProt) Secondary ID(s) of Target Chain.5,UniProt (TrEMBL) Submitted Name of Target Chain.5,UniProt (TrEMBL) Entry Name of Target Chain.5,UniProt (TrEMBL) Primary ID of Target Chain.5,BindingDB Target Chain Sequence.6,PDB ID(s) of Target Chain.6,UniProt (SwissProt) Recommended Name of Target Chain.6,UniProt (SwissProt) Entry Name of Target Chain.6,UniProt (SwissProt) Primary ID of Target Chain.6,UniProt (SwissProt) Secondary ID(s) of Target Chain.6,UniProt (TrEMBL) Submitted Name of Target Chain.6,UniProt (TrEMBL) Entry Name of Target Chain.6,UniProt (TrEMBL) Primary ID of Target Chain.6,BindingDB Target Chain Sequence.7,PDB ID(s) of Target Chain.7,UniProt (SwissProt) Recommended Name of Target Chain.7,UniProt (SwissProt) Entry Name of Target Chain.7,UniProt (SwissProt) Primary ID of Target Chain.7,UniProt (SwissProt) Secondary ID(s) of Target Ch

In [66]:
df_pre6 = df_pre5[['BindingDB Reactant_set_id','Ligand SMILES','Ligand InChI', 'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
             'Target Name','Target Source Organism According to Curator or DataSource', 'BindingDB Target Chain Sequence',
             'Target Source Organism According to Curator or DataSource']].copy()

In [67]:
df_pre6.shape

(470505, 10)

In [68]:
df_pre6.to_csv('preprocessed_data.csv')